In [ ]:
! python pretrain.py --train_data_path='wikisent2.txt'

In [ ]:
! python mlm_inference.py --inference_data_path='inference_data.txt' --model_weights_path='version_24/checkpoints/epoch=7-step=217715.ckpt'

Extension horovod.torch has not been built: /home/user/conda/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used:

In [13]:
! python finetune.py --glue_cls_task='cola' --student_weights_path='version_24/checkpoints/epoch=7-step=217715.ckpt' --epochs=3 --lr=1e-5 --batch_size=32 --finetune_distilled

tokenizer_config.json: 100% 28.0/28.0 [00:00<00:00, 168kB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 8.43MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 26.0MB/s]
config.json: 100% 570/570 [00:00<00:00, 3.62MB/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/lightning_logs
2024-01-12 17:29:32.529663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 17:29:32.529729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 17:29:32.531149: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for p

In [14]:
! python finetune.py --glue_cls_task='sst2' --student_weights_path='version_24/checkpoints/epoch=7-step=217715.ckpt' --epochs=3 --lr=1e-5 --batch_size=32 --finetune_distilled

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-01-12 17:35:06.001895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 17:35:06.001958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 17:35:06.003778: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 17:35:07.645863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Pa

Смотрим распределение классов в тестовом датасете cola

In [19]:
from argparse import Namespace
from data_modules import GLUEDataModule
from transformers import BertTokenizer
from utils import seed_everything

seed_everything(42)
params = Namespace()
params.max_length = 512
params.model_name = "bert-base-uncased"
params.glue_cls_task = 'cola'

tokenizer = BertTokenizer.from_pretrained(params.model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
data_module = GLUEDataModule(params, tokenizer)
data_module.prepare_data()
data_module.setup()

In [21]:
test_labels = []

for x in data_module.test_dataset:
    test_labels.append(x['labels'])

In [22]:
sum(test_labels) / len(test_labels)

tensor([0.3087, 0.6913])

Дообученная на COLA модель всегда предсказывают класс 1

Дообучаем bert_base_uncased

In [25]:
! python finetune.py --glue_cls_task='cola' --epochs=3 --lr=1e-5 --batch_size=8

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-01-12 18:31:48.986454: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 18:31:48.986519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 18:31:48.988528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attemp

In [8]:
! python finetune.py --glue_cls_task='sst2' --epochs=3 --lr=1e-5 --batch_size=32


libgomp: Invalid value for environment variable OMP_NUM_THREADS
Extension horovod.torch has not been built: /home/user/conda/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.

libgomp: Invalid value for environment variable OMP_NUM_THREADS
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (